In [18]:
import os

In [19]:
%pwd

'c:\\Users\\RICH-FILES\\Desktop'

In [20]:
os.chdir("../")

In [21]:
%pwd

'c:\\Users\\RICH-FILES'

In [22]:
project_dir = "C:/Users/RICH-FILES/Desktop/ml/AI-powered-Bank-Product-Recommender-Chatbot"
project_dir

'C:/Users/RICH-FILES/Desktop/ml/AI-powered-Bank-Product-Recommender-Chatbot'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_urls: str
    bank_customers: str
    product_catalog: str
    local_data_file: str
    unzip_dir: str
    
    
    
    
    
    